In [2]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict
import os

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [3]:
rr = RandomRules(
  TOTAL_NUMBERS = 35,
  CHOOSE = 5,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5],
  MAX_B_VALUES = [31, 32, 33, 34, 35]
)

In [4]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=164'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [5]:
lst = []
post_str_b1 = None
post_str_b2 = None
post_str_b3 = None
post_str_b4 = None
post_str_blast = None
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_b2)
  cols.append(post_str_b3)
  cols.append(post_str_b4)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_b2 = str_combo[1]
  post_str_b3 = str_combo[2]
  post_str_b4 = str_combo[3]
  post_str_blast = str_combo[-1]

with open('./gn_cahe_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
    'post_str_b1', 'post_str_b2', 'post_str_b3', 'post_str_b4', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))

TOTAL_LINES=7742


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""26/04/2025""","""4198""","""23 35 07 26 21""","""0721232635""","[7, 21, … 35]",7,21,23,26,35,"""07""","""21""","""23""","""26""","""35""",null,null,null,null,null,false,0,"""14_02_03_09""",-8
"""25/04/2025""","""4197""","""27 31 35 03 20""","""0320273135""","[3, 20, … 35]",3,20,27,31,35,"""03""","""20""","""27""","""31""","""35""","""07""","""21""","""23""","""26""","""35""",true,8,"""17_07_04_04""",-4
"""24/04/2025""","""4196""","""16 05 02 26 29""","""0205162629""","[2, 5, … 29]",2,5,16,26,29,"""02""","""05""","""16""","""26""","""29""","""03""","""20""","""27""","""31""","""35""",true,16,"""03_11_10_03""",3
"""23/04/2025""","""4195""","""11 33 19 20 14""","""1114192033""","[11, 14, … 33]",11,14,19,20,33,"""11""","""14""","""19""","""20""","""33""","""02""","""05""","""16""","""26""","""29""",false,0,"""03_05_01_13""",22
"""22/04/2025""","""4194""","""06 07 16 18 20""","""0607161820""","[6, 7, … 20]",6,7,16,18,20,"""06""","""07""","""16""","""18""","""20""","""11""","""14""","""19""","""20""","""33""",true,24,"""01_09_02_02""",22
"""21/04/2025""","""4193""","""08 01 07 21 25""","""0107082125""","[1, 7, … 25]",1,7,8,21,25,"""01""","""07""","""08""","""21""","""25""","""06""","""07""","""16""","""18""","""20""",true,16,"""06_01_13_04""",-38
"""20/04/2025""","""4192""","""19 10 12 28 11""","""1011121928""","[10, 11, … 28]",10,11,12,19,28,"""10""","""11""","""12""","""19""","""28""","""01""","""07""","""08""","""21""","""25""",true,24,"""01_01_07_09""",-10
"""19/04/2025""","""4191""","""10 06 15 20 21""","""0610152021""","[6, 10, … 21]",6,10,15,20,21,"""06""","""10""","""15""","""20""","""21""","""10""","""11""","""12""","""19""","""28""",true,24,"""04_05_05_01""",-3
"""18/04/2025""","""4190""","""21 23 31 05 25""","""0521232531""","[5, 21, … 31]",5,21,23,25,31,"""05""","""21""","""23""","""25""","""31""","""06""","""10""","""15""","""20""","""21""",true,10,"""16_02_02_06""",-10


In [6]:
dfRows.item(0, 'b2')

21

In [7]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str | None = None):
  if test_value == None:
    test_value = dfRows.item(0, col)
    print(f'Setting {test_value=}')

  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  
  df_10_last = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .head(10)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )
  mean_percent = df_viz['percent'].mean()
  mean_percent_75 = mean_percent * 75 / 50
  mean_percent_25 = mean_percent * 25 / 50

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart = chart * hv.HLine(mean_percent_75).opts(color='blue')
  chart = chart * hv.HLine(mean_percent_25).opts(color='blue')
  
  chart.opts(default_tools=["pan"])

  df_jumps_map = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'jumps_map'
    ).agg(pl.count('jumps_map').alias('count')) \
    .sort("count", descending=True)
  
  df_unijump = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'unijump'
    ).agg(pl.count('unijump').alias('count')) \
    .sort("count", descending=True)


  print(f'df_viz FOR {col} = {test_value}')
  display(df_viz)

  display(chart)
  

  print(f'df_10_last FOR {col} = {test_value}')
  last_results = list(set(df_10_last[post_col][0:3]))
  print(f'Last 3 results {last_results=}')
  display(df_10_last)
  
  print(f'df_jumps_map FOR {col} = {test_value}')
  display(df_jumps_map)
  # display(df_unijump)

  print(f'df_unijump FOR {col} = {test_value}')
  display(df_unijump)

  lst_post_str_b = df_viz.filter(pl.col('percent') >= mean_percent)[post_col].to_list()
  lst_jumps_map = df_jumps_map['jumps_map'].to_list() # Used to discard
  
  return lst_post_str_b, lst_jumps_map, last_results

In [8]:
possible_b1, exclude_jump_maps_b1, last_post_b1 = plot_post_col(col='str_b1', post_col='post_str_b1')

Setting test_value='07'
df_viz FOR str_b1 = 07


post_str_b1,count,percent
str,u32,f64
"""01""",78,15.354331
"""03""",60,11.811024
"""04""",56,11.023622
"""02""",54,10.629921
"""05""",45,8.858268
…,…,…
"""17""",3,0.590551
"""16""",2,0.393701
"""18""",1,0.19685


:Overlay
   .Bars.I    :Bars   [post_str_b1]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b1 = 07
Last 3 results last_results=['07', '08', '05']


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""07/04/2025""","""4179""","""08 21 22 24 07""","""0708212224""","[7, 8, … 24]",7,8,21,22,24,"""07""","""08""","""21""","""22""","""24""","""05""","""10""","""16""","""20""","""30""",true,16,"""01_13_01_02""",37
"""06/04/2025""","""4178""","""07 35 24 34 14""","""0714243435""","[7, 14, … 35]",7,14,24,34,35,"""07""","""14""","""24""","""34""","""35""","""07""","""08""","""21""","""22""","""24""",true,8,"""07_10_10_01""",-6
"""31/03/2025""","""4172""","""07 22 34 26 16""","""0716222634""","[7, 16, … 34]",7,16,22,26,34,"""07""","""16""","""22""","""26""","""34""","""08""","""12""","""13""","""25""","""33""",false,0,"""09_06_04_08""",5
"""18/03/2025""","""4159""","""07 34 16 12 21""","""0712162134""","[7, 12, … 34]",7,12,16,21,34,"""07""","""12""","""16""","""21""","""34""","""01""","""07""","""13""","""14""","""22""",false,0,"""05_04_05_13""",5
"""13/03/2025""","""4154""","""17 30 18 07 19""","""0717181930""","[7, 17, … 30]",7,17,18,19,30,"""07""","""17""","""18""","""19""","""30""","""03""","""07""","""11""","""15""","""35""",true,24,"""10_01_01_11""",1
"""10/02/2025""","""4123""","""32 07 19 25 21""","""0719212532""","[7, 19, … 32]",7,19,21,25,32,"""07""","""19""","""21""","""25""","""32""","""08""","""17""","""19""","""28""","""32""",false,0,"""12_02_04_07""",-11
"""05/01/2025""","""4087""","""28 19 07 23 20""","""0719202328""","[7, 19, … 28]",7,19,20,23,28,"""07""","""19""","""20""","""23""","""28""","""01""","""05""","""11""","""30""","""34""",false,0,"""12_01_03_05""",-13
"""20/12/2024""","""4071""","""08 22 23 07 27""","""0708222327""","[7, 8, … 27]",7,8,22,23,27,"""07""","""08""","""22""","""23""","""27""","""05""","""17""","""28""","""29""","""34""",true,16,"""01_14_01_04""",42
"""29/11/2024""","""4050""","""13 26 29 25 07""","""0713252629""","[7, 13, … 29]",7,13,25,26,29,"""07""","""13""","""25""","""26""","""29""","""11""","""13""","""25""","""29""","""35""",false,0,"""06_12_01_03""",30


df_jumps_map FOR str_b1 = 07


jumps_map,count
str,u32
"""05_01_12_06""",2
"""03_14_02_01""",2
"""04_01_09_05""",2
"""05_05_13_05""",2
"""04_09_12_01""",2
…,…
"""11_03_04_03""",1
"""10_01_08_09""",1
"""07_03_02_15""",1


df_unijump FOR str_b1 = 07


unijump,count
i64,u32
10,20
-11,17
1,14
7,13
15,12
…,…
52,1
26,1
46,1


In [9]:
possible_b2, exclude_jump_maps_b2, last_post_b2 = plot_post_col(col='str_b2', post_col='post_str_b2')

Setting test_value='21'
df_viz FOR str_b2 = 21


post_str_b2,count,percent
str,u32,f64
"""06""",15,8.62069
"""08""",13,7.471264
"""09""",12,6.896552
"""11""",11,6.321839
"""05""",11,6.321839
…,…,…
"""24""",1,0.574713
"""28""",1,0.574713
"""22""",1,0.574713


:Overlay
   .Bars.I    :Bars   [post_str_b2]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b2 = 21
Last 3 results last_results=['10', '16', '08']


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""18/04/2025""","""4190""","""21 23 31 05 25""","""0521232531""","[5, 21, … 31]",5,21,23,25,31,"""05""","""21""","""23""","""25""","""31""","""06""","""10""","""15""","""20""","""21""",true,10,"""16_02_02_06""",-10
"""16/04/2025""","""4188""","""21 13 28 29 34""","""1321282934""","[13, 21, … 34]",13,21,28,29,34,"""13""","""21""","""28""","""29""","""34""","""14""","""16""","""19""","""30""","""33""",false,0,"""08_07_01_05""",15
"""10/04/2025""","""4182""","""30 33 34 10 21""","""1021303334""","[10, 21, … 34]",10,21,30,33,34,"""10""","""21""","""30""","""33""","""34""","""01""","""08""","""09""","""10""","""12""",false,0,"""11_09_03_01""",8
"""24/03/2025""","""4165""","""32 29 27 21 06""","""0621272932""","[6, 21, … 32]",6,21,27,29,32,"""06""","""21""","""27""","""29""","""32""","""04""","""06""","""22""","""29""","""35""",false,0,"""15_06_02_03""",0
"""15/02/2025""","""4128""","""21 09 26 31 32""","""0921263132""","[9, 21, … 32]",9,21,26,31,32,"""09""","""21""","""26""","""31""","""32""","""04""","""07""","""12""","""32""","""33""",true,8,"""12_05_05_01""",-11
"""18/07/2024""","""3916""","""28 30 35 11 21""","""1121283035""","[11, 21, … 35]",11,21,28,30,35,"""11""","""21""","""28""","""30""","""35""","""01""","""15""","""16""","""33""","""35""",false,0,"""10_07_02_05""",10
"""18/05/2024""","""3855""","""12 28 21 30 33""","""1221283033""","[12, 21, … 33]",12,21,28,30,33,"""12""","""21""","""28""","""30""","""33""","""06""","""07""","""15""","""27""","""35""",false,0,"""09_07_02_03""",9
"""15/04/2024""","""3822""","""24 09 29 35 21""","""0921242935""","[9, 21, … 35]",9,21,24,29,35,"""09""","""21""","""24""","""29""","""35""","""12""","""20""","""21""","""27""","""29""",false,0,"""12_03_05_06""",-12
"""11/03/2024""","""3787""","""22 12 21 35 25""","""1221222535""","[12, 21, … 35]",12,21,22,25,35,"""12""","""21""","""22""","""25""","""35""","""06""","""12""","""14""","""19""","""35""",false,0,"""09_01_03_10""",-5


df_jumps_map FOR str_b2 = 21


jumps_map,count
str,u32
"""07_04_03_04""",2
"""19_07_03_01""",2
"""11_04_05_05""",2
"""10_07_02_05""",2
"""08_07_01_05""",2
…,…
"""12_07_01_01""",1
"""06_04_07_02""",1
"""03_10_02_01""",1


df_unijump FOR str_b2 = 21


unijump,count
i64,u32
-6,9
-7,9
-9,7
-11,7
-13,7
…,…
-24,1
18,1
22,1


In [10]:
possible_b3, exclude_jump_maps_b3, last_post_b3 = plot_post_col(col='str_b3', post_col='post_str_b3')

Setting test_value='23'
df_viz FOR str_b3 = 23


post_str_b3,count,percent
str,u32,f64
"""19""",28,7.329843
"""17""",25,6.544503
"""20""",23,6.020942
"""23""",22,5.759162
"""13""",21,5.497382
…,…,…
"""06""",5,1.308901
"""31""",5,1.308901
"""05""",4,1.04712


:Overlay
   .Bars.I    :Bars   [post_str_b3]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b3 = 23
Last 3 results last_results=['28', '16', '15']


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""18/04/2025""","""4190""","""21 23 31 05 25""","""0521232531""","[5, 21, … 31]",5,21,23,25,31,"""05""","""21""","""23""","""25""","""31""","""06""","""10""","""15""","""20""","""21""",true,10,"""16_02_02_06""",-10
"""15/04/2025""","""4187""","""29 15 23 33 10""","""1015232933""","[10, 15, … 33]",10,15,23,29,33,"""10""","""15""","""23""","""29""","""33""","""13""","""21""","""28""","""29""","""34""",false,0,"""05_08_06_04""",5
"""07/02/2025""","""4120""","""08 35 23 28 17""","""0817232835""","[8, 17, … 35]",8,17,23,28,35,"""08""","""17""","""23""","""28""","""35""","""01""","""02""","""16""","""19""","""31""",false,0,"""09_06_05_07""",1
"""04/02/2025""","""4117""","""34 35 06 23 13""","""0613233435""","[6, 13, … 35]",6,13,23,34,35,"""06""","""13""","""23""","""34""","""35""","""14""","""19""","""28""","""31""","""34""",false,0,"""07_10_11_01""",-9
"""30/01/2025""","""4112""","""23 06 22 33 34""","""0622233334""","[6, 22, … 34]",6,22,23,33,34,"""06""","""22""","""23""","""33""","""34""","""04""","""05""","""19""","""28""","""32""",true,16,"""16_01_10_01""",-42
"""15/12/2024""","""4066""","""24 31 08 23 19""","""0819232431""","[8, 19, … 31]",8,19,23,24,31,"""08""","""19""","""23""","""24""","""31""","""05""","""07""","""11""","""14""","""29""",false,0,"""11_04_01_07""",5
"""09/11/2024""","""4030""","""26 23 30 12 17""","""1217232630""","[12, 17, … 30]",12,17,23,26,30,"""12""","""17""","""23""","""26""","""30""","""02""","""08""","""09""","""27""","""29""",false,0,"""05_06_03_04""",8
"""02/11/2024""","""4023""","""08 13 23 34 29""","""0813232934""","[8, 13, … 34]",8,13,23,29,34,"""08""","""13""","""23""","""29""","""34""","""09""","""13""","""14""","""22""","""34""",false,0,"""05_10_06_05""",12
"""31/10/2024""","""4021""","""24 12 03 23 35""","""0312232435""","[3, 12, … 35]",3,12,23,24,35,"""03""","""12""","""23""","""24""","""35""","""03""","""07""","""10""","""24""","""28""",false,0,"""09_11_01_11""",32


df_jumps_map FOR str_b3 = 23


jumps_map,count
str,u32
"""05_01_05_01""",2
"""03_11_05_04""",2
"""08_01_04_02""",2
"""13_03_05_07""",1
"""15_06_07_03""",1
…,…
"""16_01_03_05""",1
"""07_06_05_07""",1
"""16_06_03_06""",1


df_unijump FOR str_b3 = 23


unijump,count
i64,u32
7,13
2,13
19,11
-1,9
1,9
…,…
-39,1
-22,1
51,1


In [11]:
possible_b4, exclude_jump_maps_b4, last_post_b4 = plot_post_col(col='str_b4', post_col='post_str_b4')

Setting test_value='26'
df_viz FOR str_b4 = 26


post_str_b4,count,percent
str,u32,f64
"""26""",44,8.782435
"""24""",35,6.986028
"""27""",33,6.586826
"""25""",32,6.387226
"""28""",32,6.387226
…,…,…
"""11""",2,0.399202
"""12""",2,0.399202
"""09""",1,0.199601


:Overlay
   .Bars.I    :Bars   [post_str_b4]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b4 = 26
Last 3 results last_results=['16', '25', '31']


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""24/04/2025""","""4196""","""16 05 02 26 29""","""0205162629""","[2, 5, … 29]",2,5,16,26,29,"""02""","""05""","""16""","""26""","""29""","""03""","""20""","""27""","""31""","""35""",true,16,"""03_11_10_03""",3
"""13/04/2025""","""4185""","""26 06 13 09 27""","""0609132627""","[6, 9, … 27]",6,9,13,26,27,"""06""","""09""","""13""","""26""","""27""","""04""","""09""","""14""","""16""","""27""",false,0,"""03_04_13_01""",-29
"""31/03/2025""","""4172""","""07 22 34 26 16""","""0716222634""","[7, 16, … 34]",7,16,22,26,34,"""07""","""16""","""22""","""26""","""34""","""08""","""12""","""13""","""25""","""33""",false,0,"""09_06_04_08""",5
"""16/03/2025""","""4157""","""26 10 21 16 30""","""1016212630""","[10, 16, … 30]",10,16,21,26,30,"""10""","""16""","""21""","""26""","""30""","""04""","""08""","""20""","""24""","""30""",true,8,"""06_05_05_04""",-2
"""10/03/2025""","""4151""","""29 06 26 15 19""","""0615192629""","[6, 15, … 29]",6,15,19,26,29,"""06""","""15""","""19""","""26""","""29""","""02""","""11""","""12""","""16""","""32""",true,16,"""09_04_07_03""",-15
"""07/03/2025""","""4148""","""26 07 03 24 30""","""0307242630""","[3, 7, … 30]",3,7,24,26,30,"""03""","""07""","""24""","""26""","""30""","""02""","""16""","""30""","""32""","""35""",true,16,"""04_17_02_04""",45
"""14/02/2025""","""4127""","""15 09 28 26 14""","""0914152628""","[9, 14, … 28]",9,14,15,26,28,"""09""","""14""","""15""","""26""","""28""","""09""","""21""","""26""","""31""","""32""",true,16,"""05_01_11_02""",-33
"""23/01/2025""","""4105""","""26 10 30 24 01""","""0110242630""","[1, 10, … 30]",1,10,24,26,30,"""01""","""10""","""24""","""26""","""30""","""03""","""07""","""15""","""28""","""31""",false,0,"""09_14_02_04""",31
"""02/12/2024""","""4053""","""05 16 03 26 31""","""0305162631""","[3, 5, … 31]",3,5,16,26,31,"""03""","""05""","""16""","""26""","""31""","""06""","""15""","""19""","""25""","""31""",false,0,"""02_11_10_05""",6


df_jumps_map FOR str_b4 = 26


jumps_map,count
str,u32
"""01_05_08_09""",2
"""03_14_02_01""",2
"""09_04_02_01""",2
"""02_03_01_09""",2
"""18_02_05_05""",2
…,…
"""22_01_02_06""",1
"""08_01_13_04""",1
"""14_08_03_04""",1


df_unijump FOR str_b4 = 26


unijump,count
i64,u32
-1,13
-2,13
-5,12
-3,11
6,11
…,…
-40,1
-42,1
36,1


In [12]:
possible_blast, exclude_jump_maps_blast, last_post_blast = plot_post_col(col='str_b5', post_col='post_str_blast')

Setting test_value='35'
df_viz FOR str_b5 = 35


post_str_blast,count,percent
str,u32,f64
"""35""",165,14.692787
"""34""",161,14.336598
"""33""",121,10.774711
"""32""",121,10.774711
"""31""",91,8.103295
…,…,…
"""14""",2,0.178094
"""17""",2,0.178094
"""13""",2,0.178094


:Overlay
   .Bars.I    :Bars   [post_str_blast]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b5 = 35
Last 3 results last_results=['35', '34', '24']


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""25/04/2025""","""4197""","""27 31 35 03 20""","""0320273135""","[3, 20, … 35]",3,20,27,31,35,"""03""","""20""","""27""","""31""","""35""","""07""","""21""","""23""","""26""","""35""",true,8,"""17_07_04_04""",-4
"""06/04/2025""","""4178""","""07 35 24 34 14""","""0714243435""","[7, 14, … 35]",7,14,24,34,35,"""07""","""14""","""24""","""34""","""35""","""07""","""08""","""21""","""22""","""24""",true,8,"""07_10_10_01""",-6
"""04/04/2025""","""4176""","""27 24 32 30 35""","""2427303235""","[24, 27, … 35]",24,27,30,32,35,"""24""","""27""","""30""","""32""","""35""","""05""","""15""","""16""","""33""","""34""",true,24,"""03_03_02_03""",3
"""30/03/2025""","""4171""","""35 05 15 29 03""","""0305152935""","[3, 5, … 35]",3,5,15,29,35,"""03""","""05""","""15""","""29""","""35""","""07""","""16""","""22""","""26""","""34""",true,2,"""02_10_14_06""",-8
"""27/03/2025""","""4168""","""27 35 05 32 08""","""0508273235""","[5, 8, … 35]",5,8,27,32,35,"""05""","""08""","""27""","""32""","""35""","""01""","""09""","""10""","""20""","""29""",true,16,"""03_19_05_03""",42
"""25/03/2025""","""4166""","""29 06 35 22 04""","""0406222935""","[4, 6, … 35]",4,6,22,29,35,"""04""","""06""","""22""","""29""","""35""","""06""","""24""","""26""","""29""","""34""",true,16,"""02_16_07_06""",31
"""14/03/2025""","""4155""","""35 11 03 07 15""","""0307111535""","[3, 7, … 35]",3,7,11,15,35,"""03""","""07""","""11""","""15""","""35""","""02""","""05""","""15""","""16""","""27""",true,26,"""04_04_04_20""",16
"""12/03/2025""","""4153""","""28 14 35 03 15""","""0314152835""","[3, 14, … 35]",3,14,15,28,35,"""03""","""14""","""15""","""28""","""35""","""07""","""17""","""18""","""19""","""30""",false,0,"""11_01_13_07""",-40
"""08/03/2025""","""4149""","""02 35 30 32 16""","""0216303235""","[2, 16, … 35]",2,16,30,32,35,"""02""","""16""","""30""","""32""","""35""","""02""","""04""","""15""","""16""","""31""",true,8,"""14_14_02_03""",25


df_jumps_map FOR str_b5 = 35


jumps_map,count
str,u32
"""01_06_14_02""",3
"""03_03_19_08""",2
"""03_04_12_10""",2
"""08_01_01_18""",2
"""04_04_17_05""",2
…,…
"""01_20_09_03""",1
"""01_13_07_09""",1
"""02_11_03_09""",1


df_unijump FOR str_b5 = 35


unijump,count
i64,u32
10,28
-4,24
-1,23
-7,22
12,21
…,…
76,1
38,1
56,1


In [13]:
b_count = rr.CHOOSE
min_b = rr.MIN_B_VALUES[0]
max_b = rr.MAX_B_VALUES[-1]


combos = combinations(range(min_b, max_b + 1), b_count)
# print("generating combos: ", len(list(combos)))

In [14]:
parquet_file = './parquet_gn_v3.parquet'

# check if parquet_file exists
if not os.path.exists(parquet_file):
  print(f'{parquet_file=} does not exists. creating')
  lst = []
  post_str_b1 = None
  post_str_blast = None
  for row in combos:
    cols = []
    str_row = " ".join([str(r).zfill(2) for r in row])
    id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=str_row)
    cols.append(id)
    cols.append(bolillas_int)
    cols.extend([b for b in bolillas_int])
    cols.extend(str_combo)
    cols.append(post_str_b1)
    cols.append(post_str_blast)

    is_rare, reason = rr.is_rare_combination(bolillas_int, id)
    cols.append(is_rare)
    cols.append(reason)

    cols.append(rr.jumps_map(bolillas_int))
    cols.append(rr.unijump(bolillas_int))
    
    lst.append(cols)

    post_str_b1 = str_combo[0]
    post_str_blast = str_combo[-1]

  with open('./gn_cahe_v3.json', 'w') as file: 
    json.dump(lst, file)

  df_Orows = pl.DataFrame(
    lst,
    schema=[
      # 'fecha', 'sorteo', 'bolillas',
      'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
      'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
      'post_str_b1', 'post_str_blast',
      'is_rare_combination', 'reason',
      "jumps_map", "unijump"
    ],
    schema_overrides={
      'combo': pl.Array(pl.UInt8, 5),
      'b1': pl.UInt8,
      'b2': pl.UInt8,
      'b3': pl.UInt8,
      'b4': pl.UInt8,
      'b5': pl.UInt8,
      'b6': pl.UInt8,
      'reason': pl.UInt8
    },
    orient='row'
  )

  df_Orows.write_parquet(parquet_file)
  TOTAL_LINES = len(df_Orows)
  print(f'{TOTAL_LINES=} loaded to {parquet_file=}')

  display(df_Orows.head(10))
else:
  df_Orows = pl.read_parquet(parquet_file)

In [15]:
# possible_b1, exclude_jump_maps_b1
df_possible = df_Orows.filter(
  pl.col("str_b1").is_in(possible_b1),
  pl.col("str_b2").is_in(possible_b2),
  pl.col("str_b3").is_in(possible_b3),
  pl.col("str_b4").is_in(possible_b4),
  pl.col("str_b5").is_in(possible_blast),
  
  ~pl.col("str_b1").is_in(last_post_b1),
  ~pl.col("str_b2").is_in(last_post_b2),
  ~pl.col("str_b3").is_in(last_post_b3),
  ~pl.col("str_b4").is_in(last_post_b4),
  ~pl.col("str_b5").is_in(last_post_blast),

  ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_b2 + exclude_jump_maps_b3 + exclude_jump_maps_b4 + exclude_jump_maps_blast),
  pl.col("is_rare_combination") == False
)


display(df_possible)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0105111832""","[1, 5, … 32]",1,5,11,18,32,"""01""","""05""","""11""","""18""","""32""","""01""","""31""",false,0,"""04_06_07_14""",7
"""0105111932""","[1, 5, … 32]",1,5,11,19,32,"""01""","""05""","""11""","""19""","""32""","""01""","""31""",false,0,"""04_06_08_13""",3
"""0105112031""","[1, 5, … 31]",1,5,11,20,31,"""01""","""05""","""11""","""20""","""31""","""01""","""30""",false,0,"""04_06_09_11""",-2
"""0105112032""","[1, 5, … 32]",1,5,11,20,32,"""01""","""05""","""11""","""20""","""32""","""01""","""31""",false,0,"""04_06_09_12""",-1
"""0105112033""","[1, 5, … 33]",1,5,11,20,33,"""01""","""05""","""11""","""20""","""33""","""01""","""32""",false,0,"""04_06_09_13""",0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""0919242830""","[9, 19, … 30]",9,19,24,28,30,"""09""","""19""","""24""","""28""","""30""","""09""","""29""",false,0,"""10_05_04_02""",-5
"""0919242831""","[9, 19, … 31]",9,19,24,28,31,"""09""","""19""","""24""","""28""","""31""","""09""","""30""",false,0,"""10_05_04_03""",-4
"""0919242833""","[9, 19, … 33]",9,19,24,28,33,"""09""","""19""","""24""","""28""","""33""","""09""","""32""",false,0,"""10_05_04_05""",-2


In [16]:
df_possible.sample(n=1)


id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0115182730""","[1, 15, … 30]",1,15,18,27,30,"""01""","""15""","""18""","""27""","""30""","""01""","""29""",false,0,"""14_03_09_03""",-29
